In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam


In [ ]:
# data_dir = 'Objection_detection/Test_img'
# img_height = 32
# img_width = 32
# batch_size = 9   # n개의 이미지만 학습

# train_ds = tf.keras.utils.image_dataset_from_directory(
#     data_dir,                      
#     validation_split=0.2,               
#     subset="training",                   
#     label_mode='int',                    
#     batch_size=batch_size,               
#     shuffle=True,                       
#     seed= 32,                            
#     image_size=(img_height, img_width),  
#     color_mode='rgb',                    
#     )

# val_ds = tf.keras.utils.image_dataset_from_directory(
#     data_dir, 					               
#     validation_split=0.2,				      
#     subset="validation",				        
#     seed= 32,					                
#     image_size=(img_height, img_width),				
#     batch_size=batch_size,
#     color_mode='rgb')	

In [ ]:
# from PIL import Image
# import os

# image_dir = 'Test_img/img'
# label_dir = 'Test_img/img_label'


In [ ]:

# image_files = os.listdir(image_dir)
# if image_files:
#     image_path = os.path.join(image_dir, image_files[0])  # 첫 번째 파일 선택
#     image = Image.open(image_path)  # 이미지 불러오기
#     image.show()  # 이미지 보기 (새 창으로 열림)
# else:
#     print("이미지 파일이 없습니다.")

In [5]:
# label_dir = 'Test_img/img_label'

# # 디렉토리 내의 첫 번째 레이블 파일 선택
# label_files = os.listdir(label_dir)
# if label_files:
#     label_path = os.path.join(label_dir, label_files[0])  # 첫 번째 레이블 파일 선택
    
#     # 레이블 파일 읽기
#     with open(label_path, 'r') as f:
#         labels = f.readlines()  # 파일의 모든 줄을 읽기
        
#     # 레이블 데이터 출력 (클래스 번호, x_center, y_center, width, height)
#     for label in labels:
#         values = label.strip().split()  # 공백으로 분리
#         class_id = int(values[0])  # 클래스 번호
#         x_center, y_center, width, height = map(float, values[1:])  # 좌표와 크기
#         print(f"Class: {class_id}, x_center: {x_center}, y_center: {y_center}, width: {width}, height: {height}")
# else:
#     print("레이블 파일이 없습니다.")

Class: 0, x_center: 0.61284, y_center: 0.670039, width: 0.22179, height: 0.60856
Class: 0, x_center: 0.410182, y_center: 0.669455, width: 0.164073, height: 0.552918


In [8]:
import os
import tensorflow as tf
import numpy as np

# 이미지 디렉토리와 레이블 디렉토리 경로 설정
image_dir = 'Test_img/img'
label_dir = 'Test_img/img_label'

# YOLO 레이블 파일에서 정규화된 바운딩 박스 좌표를 읽는 함수
def load_labels(label_file):
    boxes = []
    with open(label_file, 'r') as f:
        for line in f.readlines():
            data = line.strip().split()
            # YOLO 형식: class_id, x_center, y_center, width, height
            class_id, x_center, y_center, width, height = map(float, data)
            boxes.append([class_id, x_center, y_center, width, height])  # 정규화된 좌표를 그대로 사용
    return boxes

# 이미지 및 레이블 데이터를 로드하는 함수
def load_data(image_dir, label_dir, img_width, img_height, max_boxes=10):
    image_files = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.jpg')])
    label_files = sorted([os.path.join(label_dir, f.replace('.jpg', '.txt')) for f in os.listdir(image_dir) if f.endswith('.jpg')])

    images = []
    labels = []
    missing_labels = 0

    for img_file, lbl_file in zip(image_files, label_files):
        # 레이블 파일 존재 여부 확인
        if not os.path.exists(lbl_file):
            print(f"Label file not found: {lbl_file}")
            missing_labels += 1
            continue
        
        # 이미지 읽기
        img = tf.keras.preprocessing.image.load_img(img_file, target_size=(img_height, img_width))
        img = tf.keras.preprocessing.image.img_to_array(img)
        images.append(img)

        # 레이블 읽기
        boxes = load_labels(lbl_file)
        
        # 바운딩 박스 수가 max_boxes보다 적은 경우, None으로 패딩
        while len(boxes) < max_boxes:
            boxes.append([0, 0, 0, 0, 0])  # 클래스 ID와 좌표를 0으로 채움

        labels.append(boxes[:max_boxes])  # max_boxes로 잘라냄

    print(f"Missing label files: {missing_labels}")
    return np.array(images), np.array(labels)

# 이미지 및 레이블 로드
img_width = 32
img_height = 32

train_images, train_labels = load_data(image_dir, label_dir, img_width, img_height)

# 텐서플로 데이터셋으로 변환
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(32)

Missing label files: 0


In [12]:
val_size = int(0.2 * tf.data.experimental.cardinality(train_ds).numpy())  # 검증 데이터 크기 계산
batch_size = 1

# train_ds를 검증 데이터와 훈련 데이터로 나누기
train_ds = train_ds.skip(val_size)  # 남은 훈련 데이터
val_ds = train_ds.take(val_size)  # 검증 데이터

In [17]:
import tensorflow as tf

# CNN 모델 정의
def create_cnn_model(input_shape, max_boxes):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(max_boxes * 5)  # max_boxes 수에 맞게 출력 차원 설정
    ])
    return model

# 커스텀 레이어로 출력을 reshape ???
class ReshapeLayer(tf.keras.layers.Layer):
    def __init__(self, target_shape):
        super(ReshapeLayer, self).__init__()
        self.target_shape = target_shape

    def call(self, inputs):
        return tf.reshape(inputs, (-1, *self.target_shape))

# 모델 생성
img_height, img_width = 32, 32
max_boxes = 10  # 최대 바운딩 박스 수
input_shape = (img_height, img_width, 3)  # RGB 이미지

model = create_cnn_model(input_shape, max_boxes)

# reshape 레이어 추가
model.add(ReshapeLayer((max_boxes, 5)))  # 출력 형태를 (batch_size, max_boxes, 5)로 변경

# 모델 컴파일
model.compile(optimizer='adam', 
              loss=tf.keras.losses.MeanSquaredError(),  # MSE 사용
              metrics=['accuracy'])

# 모델 학습
model.fit(train_ds, validation_data=val_ds, epochs=300)  # 에포크 수는 조정 가능

# 학습 완료 메시지 출력
print("훈련 완료!")


Epoch 1/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.1668 - loss: 295.3409
Epoch 2/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2017 - loss: 24.6452
Epoch 3/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.2592 - loss: 8.2605 
Epoch 4/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2335 - loss: 2.9671
Epoch 5/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2597 - loss: 1.5455
Epoch 6/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3088 - loss: 0.9727
Epoch 7/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3247 - loss: 0.6250
Epoch 8/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3284 - loss: 0.4229
Epoch 9/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3280 - loss: 0.3221
Epoch 10/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.3181 - loss: 0.2643
Epoch 11/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3127 - loss: 0.2212 
Epoch 12/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.3117 

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
import os
import tensorflow as tf
import numpy as np

# 이미지 및 레이블 디렉토리 경로 설정
image_dir = 'Test_img/img'
label_dir = 'Test_img/img_label'

# YOLO 레이블 파일에서 정규화된 바운딩 박스 좌표를 읽는 함수
def load_labels(label_file):
    boxes = []
    with open(label_file, 'r') as f:
        for line in f.readlines():
            data = list(map(float, line.strip().split()))
            boxes.append(data)  # 정규화된 좌표를 그대로 사용
    return boxes

# 이미지 및 레이블 데이터를 로드하는 함수
def load_data(image_dir, label_dir, img_width, img_height, max_boxes=10):
    image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.jpg')])
    images, labels = [], []
    missing_labels = 0

    for img_file in image_files:
        lbl_file = os.path.join(label_dir, img_file.replace('.jpg', '.txt'))
        if not os.path.exists(lbl_file):
            print(f"Label file not found: {lbl_file}")
            missing_labels += 1
            continue
        
        img = tf.keras.preprocessing.image.load_img(os.path.join(image_dir, img_file), target_size=(img_height, img_width))
        images.append(tf.keras.preprocessing.image.img_to_array(img))
        boxes = load_labels(lbl_file)

        # 바운딩 박스 수가 max_boxes보다 적은 경우, None으로 패딩
        boxes += [[0, 0, 0, 0, 0]] * (max_boxes - len(boxes))  # 패딩 추가
        labels.append(boxes[:max_boxes])  # max_boxes로 잘라냄

    print(f"Missing label files: {missing_labels}")
    return np.array(images), np.array(labels)

# 이미지 및 레이블 로드
img_width, img_height, max_boxes = 32, 32, 10
train_images, train_labels = load_data(image_dir, label_dir, img_width, img_height)

# 텐서플로 데이터셋으로 변환
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(32)
val_size = int(0.2 * tf.data.experimental.cardinality(train_ds).numpy())

# train_ds를 검증 데이터와 훈련 데이터로 나누기
train_ds, val_ds = train_ds.skip(val_size), train_ds.take(val_size)

# CNN 모델 정의
def create_cnn_model(input_shape, max_boxes):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(max_boxes * 5)  
    ])
    return model

# 모델 생성 및 컴파일
model = create_cnn_model((img_height, img_width, 3), max_boxes)
model.add(tf.keras.layers.Reshape((max_boxes, 5)))  # 출력 형태를 (batch_size, max_boxes, 5)로 변경
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# 모델 학습
model.fit(train_ds, validation_data=val_ds, epochs=300)

# 학습 완료 메시지 출력
print("훈련 완료!")

### CNN 모델

<!-- # import tensorflow as tf
# from tensorflow.keras import layers, models

# # CNN 모델 정의
# model = models.Sequential()
# # 컨볼루션 레이어 추가: 32개 필터, 3x3 커널 크기, relu 활성화 함수
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
# # 맥스풀링 레이어 추가: 2x2 풀링 크기
# model.add(layers.MaxPooling2D((2, 2)))
# # 모델 정규화
# model.add(layers.BatchNormalization())

# model.add(layers.Dropout(0.25))  # 드롭아웃 비율 25%

# # 추가 컨볼루션 레이어: 64개 필터
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# # 맥스풀링 레이어
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.BatchNormalization())

# # 추가 컨볼루션 레이어: 64개 필터
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# # 모델 정규화
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.5))  # 드롭아웃 비율 25%

# # 플래튼 레이어로 차원 축소
# model.add(layers.Flatten())
# # 밀집 연결 레이어(Dense): 64 유닛, relu 활성화
# model.add(layers.Dense(64, activation='relu'))
# # 출력 레이어: 클래스 수만큼 유닛, 소프트맥스 활성화 함수로 클래스 확률 출력
# model.add(layers.Dense(4, activation='softmax'))



# # 모델 컴파일: 옵티마이저, 손실 함수, 평가 지표 설정
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
# model.compile(optimizer=optimizer,
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])


# # 모델 구조 요약 출력
# model.summary() -->